## Создание базы данных и таблиц

Логика, по которой программа работает с базой данных, в общем виде выглядит так:

1. Подключиться к базе данных, «создать соединение».

2. Подготовить SQL-запросы.

3. Выполнить запросы.

4. Закрыть соединение с БД.

Например, вот как будет выглядеть код для создания таблиц `video_products` и `directors` в файле базы данных *db.sqlite*:

In [1]:
import sqlite3

# Если в текущей директории нет файла db.sqlite - 
# он будет создан; одновременно будет создано и соединение с базой данных.
# Если файл существует, функция connect просто подключится к базе.
con = sqlite3.connect('db.sqlite')

# Создаём специальный объект cursor для работы с БД.
# Вся дальнейшая работа будет вестись через методы этого объекта.
cur = con.cursor()

# Готовим SQL-запросы.
# Для читаемости запрос обрамлён в тройные кавычки и разбит построчно.
query_1 = '''
CREATE TABLE IF NOT EXISTS directors(
    id INTEGER PRIMARY KEY,
    full_name TEXT,
    birth_year INTEGER
);
'''
query_2 = '''
CREATE TABLE IF NOT EXISTS video_products(
    id INTEGER PRIMARY KEY,
    title TEXT,
    product_type TEXT,
    release_year INTEGER
);
'''

# Применяем запросы.
cur.execute(query_1)
cur.execute(query_2)

# Закрываем соединение с БД.
con.close()

* Инструкция `con = sqlite3.connect('db.sqlite')` создаёт подключение к базе данных, указанной в аргументе функции `.connect()`. Если по заданному пути нет файла базы данных с таким именем — никакой ошибки не возникнет: файл базы данных будет создан автоматически.

* Метод `execute()` принимает одиночный SQL-запрос в виде python-строки, заключённой в тройные кавычки. Нужно создать две таблицы — значит, вызываем метод `execute()` дважды.

* Условие `IF NOT EXISTS` проверяет, нет ли в БД таблицы с переданным в запросе названием. Если такой таблицы нет — сработает инструкция `CREATE`, и таблица будет создана.

> В базе данных SQLite не может быть двух таблиц с одинаковым названием. Поэтому перед созданием таблицы нужно установить условие `IF NOT EXISTS`. В результате новая таблица будет создана только в том случае, если таблицы с таким именем в базе нет.

***
## Резидентные базы

Базу данных можно создать не в файле, а в памяти компьютера; такие базы называют резидентными.

Резидентная база создаётся так:

```py
con = sqlite3.connect(':memory:')
```

После закрытия соединения резидентная база данных удаляется из памяти. Такие базы могут быть полезны для отладки и тестирования кода.

***
## Несколько SQL-запросов сразу

Для одновременного выполнения нескольких SQL-запросов в модуле sqlite3 есть метод `executescript()`. С его помощью запросы на создание двух таблиц можно записать компактнее:

```py
...

query = '''
CREATE TABLE IF NOT EXISTS directors(
    id INTEGER PRIMARY KEY,
    full_name TEXT,
    birth_year INTEGER
);

CREATE TABLE IF NOT EXISTS video_products(
    id INTEGER PRIMARY KEY,
    title TEXT,
    product_type TEXT,
    release_year INTEGER
);
'''

cur.executescript(query)

...
```

***
## Заполнение таблицы: INSERT

Теперь заполним таблицу данными; сделаем это с помощью того же метода `execute()`, но на этот раз передадим в него команду `INSERT` с нужными данными:

In [2]:
...
con = sqlite3.connect('db.sqlite')
cur = con.cursor()

query = '''
    INSERT INTO video_products(id, title, product_type, release_year)
    VALUES (1, 'Весёлые мелодии', 'Мультсериал', 1930);
'''
cur.execute(query)
con.commit()
con.close()

В метод `execute()` можно было бы передать и обычную f-строку с соответствующими параметрами, однако такой синтаксис небезопасен и может стать причиной взлома через механизм [SQL-инъекций](https://docs.python.org/3/library/sqlite3.html#how-to-use-placeholders-to-bind-values-in-sql-queries).

Для защиты от попыток взлома используется специальный синтаксис: в метод `execute()` значения полей можно передавать в виде кортежа.

Для этого:

* в первом аргументе передаётся запрос `INSERT` с количеством ячеек таблицы; ячейки обозначаются вопросительными знаками через запятую; например, четыре вопросительных знака обозначают четыре ячейки;

* во втором аргументе передаётся кортеж со значениями полей. Модуль SQLite поймёт всё с полуслова и разместит значения в соответствующие ячейки.

Добавим в таблицу `video_products` новую запись. В таблице четыре поля; значит, в первом аргументе передаём запрос, где после `VALUES` — четыре вопросительных знака:

In [3]:
con = sqlite3.connect('db.sqlite')
cur = con.cursor()

cur.execute(
    'INSERT INTO video_products VALUES(?, ?, ?, ?);',
    (2, 'Весёлая мелодия', 'Мультсериал', 1930)
)

con.commit()
con.close()

Данные будут добавлены, а лазейка для злоумышленников будет закрыта.

Кроме того, с помощью подобного синтаксиса можно добавить несколько записей сразу, для этого применяют метод-shortcut `executemany()`.

Первым аргументом в метод `executemany()` передаётся запрос с указанием количества заполняемых ячеек, а вторым — список, каждый элемент которого — это кортеж со значениями полей. 

Этот способ позволяет быстро и просто создать несколько записей: 

In [4]:
con = sqlite3.connect('db.sqlite')
cur = con.cursor()

directors = [
    (1, 'Текс Эйвери', 1908),
    (2, 'Роберт Земекис', 1952),
    (3, 'Джерри Чиникей', 1912),
]
video_products = [
    (3, 'Весёлые мелодии', 'Мультсериал', 1930),
    (4, 'Кто подставил кролика Роджера', 'Фильм', 1988),
    (5, 'Безумные мелодии Луни Тюнз', 'Мультсериал', 1931),
    (6, 'Розовая пантера: Контроль за вредителями', 'Мультфильм', 1969)
]

cur.executemany('INSERT INTO directors VALUES(?, ?, ?);', directors)
cur.executemany('INSERT INTO video_products VALUES(?, ?, ?, ?);', video_products)

con.commit()
con.close()

> При выполнении запросов на добавление или изменение данных в таблицах, например, при выполнении инструкции `INSERT`, должен вызываться метод `commit()`; если не вызвать этот метод — изменения не будут сохранены в базе данных.

In [ ]:
import sqlite3

con = sqlite3.connect('db.sqlite')
cur = con.cursor()

query_1 = '''
CREATE TABLE IF NOT EXISTS ice_cream(
    title TEXT,
    description TEXT,
    category TEXT
);
'''

ice_cream = [
    ('Классический пломбир',
     'Настоящее мороженое, для истинных ценителей вкуса. Если на столе появляется пломбир — это не надолго.',
     'Обычное',
     ),
    ('Мороженое с кузнечиками',
     'В колумбийском стиле: с добавлением карамелизованных кузнечиков.',
     'Экзотическое',
     ),
    ('Мороженое со вкусом сыра чеддер',
     'Вкус настоящего сыра в вафельном стаканчике.',
     'Экзотическое',
     ),
    ('Пломбир 1937',
     'Пломбир по рецепту 1937 года Московского хладокомбината',
     'Обычное'
     ),
]

cur.executemany('INSERT INTO ice_cream VALUES(?, ?, ?);', ice_cream)

cur.execute(query_1)
con.close()